Author: [vqoley](https://github.com/vqoley/vgc---google-colab-bulk-downloader-to-google-drive)

1. Install google-auth lib

In [ ]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2
!google-auth


In [ ]:
from google.colab import auth
auth.authenticate_user()

2. Go to folder that you wish all downloaded file will be saved/keep.

In [ ]:
cd gdrive/MyDrive/

/content/gdrive/MyDrive


3. Run script below

*   select path that where you will save *.txt file then press enter button on your keyboard
*   click button upload to upload your *.txt file. Make sure inside *.txt file have url line by line and then wait for Start Download button appear
*   Click start button to start download 







In [ ]:
from google.colab import drive
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload, MediaIoBaseUpload
from ipywidgets import FileUpload, Button, VBox, HBox
from IPython.display import display
from googleapiclient.http import MediaFileUpload
import io
import os
import re
import requests
import threading
import urllib.request
import requests
from tqdm import tqdm
from ipywidgets import Button, Output

def main():
  drive.mount('/content/gdrive')
  global service
  service = build('drive', 'v3')
  global drive_path_input
  drive_path_input = input('Enter the path in Google Drive where you want to save the downloaded files (e.g. "folder/subfolder"): ')
  global upload_button
  upload_button = FileUpload(accept='.txt', multiple=False)
  global upload_output
  upload_output = VBox([upload_button])
  global file_id
  file_id = None
  print('Upload a text file containing the URLs to download:')
  display(upload_output)
  upload_button.observe(on_upload_button_click, names='value')

def on_upload_button_click(change):
  global file_id
  global urls
  global download_button
  global progress_output
  file_name = list(upload_button.value.keys())[0]
  file_content = upload_button.value[file_name]['content']
  file_metadata = {'name': file_name, 'parents':[get_folder_id_by_path(drive_path_input)]}
  media = MediaIoBaseUpload(io.BytesIO(file_content), mimetype='text/plain', resumable=True)
  file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
  file_id = file.get('id')
  urls = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', file_content.decode('utf-8'))
  download_button = Button(description='Start Download')
  download_button.on_click(on_download_button_click)
  progress_output = VBox([])
  upload_output.children = [VBox([download_button, progress_output])]

def on_download_button_click(change):
    global urls
    global progress_output
    progress_output.children = [Button(description='Downloading...', disabled=True)]
    for url in urls:
        filename = os.path.basename(url)
        print("Downloading... ",filename, end='')
        response = requests.get(url, stream=True) 
        total_size = int(response.headers.get('content-length', 0))
        block_size = 1024
        progress_bar = tqdm(total=total_size, unit='iB', unit_scale=True)
        #print("\rDownloading... {filename}", end='')
        with open(filename, 'wb') as file:
            for data in response.iter_content(block_size):
                progress_bar.update(len(data))
                file.write(data)
                progress = f"{file.tell() / total_size:.1%}" if total_size > 0 else "?"
                threading.Thread(target=update_progress, args=(filename, progress)).start()
        threading.Thread(target=download_and_upload_to_drive, args=(url, filename)).start()
        progress_bar.close()



def download_and_upload_to_drive(url, filename):
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        total_size = int(r.headers.get('content-length', 0))
        bytes_so_far = 0
        with open(filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                bytes_so_far += len(chunk)
                f.write(chunk)
                progress = f"{bytes_so_far / total_size:.1%}" if total_size > 0 else "?"
               # print(f"\r{filename}: {progress}", end="")
                threading.Thread(target=update_progress, args=(filename, progress)).start()

    file_metadata = {'name': filename, 'parents': [get_folder_id_by_path(drive_path_input)]}
    media = MediaFileUpload(filename, resumable=True)
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()

    # Move the file to the specified folder
    file_id = file.get('id')
    file_metadata = {'addParents': [get_folder_id_by_path(drive_path_input)]}
    service.files().update(fileId=file_id, body=file_metadata).execute()


def update_progress(filename, progress):
  for item in progress_output.children:
    if item.description.startswith(filename):
      item.description = f'{filename} ({progress})'
      #print(f"\r{filename}: {progress}", end="")
      progress_text = f"Uploading {filename}: {progress} complete"
      progress_output.children = [Output(value=progress_text)]


def download_file(url, filename):
  with requests.get(url, stream=True) as r:
    r.raise_for_status()
    total_size = int(r.headers.get('content-length', 0))
    bytes_so_far = 0
    with open(filename, 'wb') as f:
      for chunk in r.iter_content(chunk_size=8192):
        bytes_so_far += len(chunk)
        f.write(chunk)
        progress = f"{bytes_so_far / total_size:.1%}" if total_size > 0 else "?"
        threading.Thread(target=update_progress, args=(filename, progress)).start()
  file_metadata = {'name': filename, 'parents':[get_folder_id_by_path(drive_path_input)]}
  media = MediaFileUpload(filename, resumable=True)
  file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
  # Move the file to the specified folder
  file_id = file.get('id')
  file_metadata = {'parents': [get_folder_id_by_path(drive_path_input)]}
  service.files().update(fileId=file_id, body=file_metadata).execute()

def start_download():
    urls = get_urls(text.get("1.0", tk.END))
    for url in urls:
        t = threading.Thread(target=download_file, args=(url, drive_path))
        t.start()


def on_start_button_click(b):
    global urls
    with open(uploaded_file_path.value, 'r') as f:
        urls = f.readlines()
    for url in urls:
        url = url.strip()
        if url:
            filename = os.path.basename(url)
            t = threading.Thread(target=download_file, args=(url, filename, drive_path.value))
            t.start()
            threads.append(t)


def progress_callback(block_num, block_size, total_size):
    percentage = 100.0 * block_num * block_size / total_size
    print(f"{percentage:.1f}% downloaded")

def get_folder_id_by_path(path):
  if path == "":
    return "root"
  else:
    try:
      path_list = path.split("/")
      current_folder = "root"
      for folder_name in path_list:
        if folder_name != "":
          query = "mimeType='application/vnd.google-apps.folder' and trashed = false and name = '"+folder_name+"' and '"+current_folder+"' in parents"
          results = service.files().list(q=query,fields="nextPageToken, files(id, name)").execute()
          items = results.get("files", [])
          if items:
            current_folder = items[0]["id"]
          else:
            file_metadata = {'name': folder_name, 'parents':[current_folder],'mimeType':'application/vnd.google-apps.folder'}
            folder = service.files().create(body=file_metadata, fields='id').execute()
            current_folder = folder.get('id')
        else:
          continue
      return current_folder
    except HttpError as error:
      print(f"An error occurred: {error}")
      return None

if __name__ == '__main__':
    main()

